<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [6]:
## Task 1: Mark all launch sites on a map
## Task 1: Mark all launch sites on a map

# First, let's download the dataset directly
import requests
import io

# URL for the SpaceX dataset
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv"

# Download the data
response = requests.get(url)
spacex_df = pd.read_csv(io.StringIO(response.text))

# Extract launch site locations
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()
launch_sites.columns = ['Launch Site', 'Latitude', 'Longitude']  # Rename for consistency

# Create a map centered around the mean of launch site coordinates
map_center = [launch_sites['Latitude'].mean(), launch_sites['Longitude'].mean()]
launch_map = folium.Map(location=map_center, zoom_start=4)

# Create a feature group for launch sites
launch_sites_layer = folium.FeatureGroup(name='Launch Sites')

# Add each launch site to the map
for index, site in launch_sites.iterrows():
    # Create a marker for each launch site
    marker = folium.Marker(
        location=[site['Latitude'], site['Longitude']],
        popup=site['Launch Site'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    )
    marker.add_to(launch_sites_layer)

# Add the launch sites layer to the map
launch_sites_layer.add_to(launch_map)

# Add layer control to toggle launch sites
folium.LayerControl().add_to(launch_map)

# Display the map
launch_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [7]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [12]:
# Task 2: Mark the success/failed launches for each site on the map
# Extract launch records with outcomes
launch_records = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]  # 'class' = 1 (success), 0 (failure)
launch_records.columns = ['Launch Site', 'Latitude', 'Longitude', 'Outcome']  # Rename columns for clarity
# Create a base map centered on all launch sites
map_center = [launch_sites['Latitude'].mean(), launch_sites['Longitude'].mean()]
outcome_map = folium.Map(location=map_center, zoom_start=4)

# Create a MarkerCluster layer for better visualization of dense points
marker_cluster = MarkerCluster().add_to(outcome_map)

# Color mapping: green for success (class=1), red for failure (class=0)
def get_marker_color(outcome):
    return 'green' if outcome == 1 else 'red'

# Add each launch record to the map
for index, record in launch_records.iterrows():
    folium.Marker(
        location=[record['Latitude'], record['Longitude']],
        popup=f"Site: {record['Launch Site']}<br>Outcome: {'Success' if record['Outcome'] == 1 else 'Failure'}",
        icon=folium.Icon(color=get_marker_color(record['Outcome']), icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Add layer control (optional)
folium.LayerControl().add_to(outcome_map)

# Display the map
outcome_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [14]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [15]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Add a new column 'marker_color' based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Verify the new column
print(spacex_df[['class', 'marker_color']].head())
import numpy as np

# Using numpy where() for better performance with large datasets
spacex_df['marker_color'] = np.where(spacex_df['class'] == 1, 'green', 'red')

# Verify
print(spacex_df[['class', 'marker_color']].sample(5))
# Create the map
outcome_map = folium.Map(location=[launch_sites['Latitude'].mean(), 
                                  launch_sites['Longitude'].mean()], 
                        zoom_start=4)

# Create marker cluster
marker_cluster = MarkerCluster().add_to(outcome_map)

# Add markers using the pre-computed colors
for index, record in spacex_df.iterrows():
    folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"""
            Site: {record['Launch Site']}<br>
            Mission: {record.get('Mission', 'N/A')}<br>
            Outcome: {'Success' if record['class'] == 1 else 'Failure'}
        """,
        icon=folium.Icon(color=record['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
outcome_map

   class marker_color
0      0          red
1      0          red
2      0          red
3      0          red
4      0          red
    class marker_color
6       0          red
35      0          red
15      0          red
10      0          red
16      0          red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [19]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

# Save the map
outcome_map.save('launch_sites_map.html')

# Display download link
from IPython.display import FileLink
FileLink('launch_sites_map.html')

/drive/DS0203EN/module_3/launch_sites_map.html

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [20]:
# TASK 3: Calculate the distances between a launch site to its proximities
# Proximities data (manually collected)
proximities = {
    "CCAFS LC-40": {
        "Coastline": (28.56367, -80.57163),
        "Cape Canaveral City": (28.4058, -80.6049),
        "Kennedy Space Center": (28.5852, -80.6508)
    },
    "CCAFS SLC-40": {
        "Coastline": (28.56367, -80.57163),
        "Cape Canaveral City": (28.4058, -80.6049),
        "Kennedy Space Center": (28.5852, -80.6508)
    },
    "KSC LC-39A": {
        "Coastline": (28.5801, -80.6422),
        "Titusville City": (28.6122, -80.8076),
        "Kennedy Space Center": (28.5852, -80.6508)
    },
    "VAFB SLC-4E": {
        "Coastline": (34.6329, -120.6266),
        "Lompoc City": (34.6391, -120.4579),
        "Vandenberg AFB HQ": (34.7483, -120.5182)
    }
}



def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance (in km) between two points
    on Earth using their latitude and longitude in decimal degrees.
    """
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

# Create a DataFrame to store distances
distance_data = []

for site, coords in proximities.items():
    site_lat = spacex_df[spacex_df['Launch Site'] == site]['Lat'].iloc[0]
    site_lon = spacex_df[spacex_df['Launch Site'] == site]['Long'].iloc[0]
    
    for location, (lat, lon) in coords.items():
        distance = haversine_distance(site_lat, site_lon, lat, lon)
        distance_data.append({
            'Launch Site': site,
            'Proximity': location,
            'Distance (km)': distance
        })

distances_df = pd.DataFrame(distance_data)
print(distances_df)
# Create a map centered on all launch sites
proximity_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=4)

# Add launch sites (green)
for site in spacex_df['Launch Site'].unique():
    site_data = spacex_df[spacex_df['Launch Site'] == site].iloc[0]
    folium.Marker(
        location=[site_data['Lat'], site_data['Long']],
        popup=f"<b>{site}</b>",
        icon=folium.Icon(color='green', icon='rocket', prefix='fa')
    ).add_to(proximity_map)

# Add proximities (blue) with distance labels
for site, coords in proximities.items():
    site_lat = spacex_df[spacex_df['Launch Site'] == site]['Lat'].iloc[0]
    site_lon = spacex_df[spacex_df['Launch Site'] == site]['Long'].iloc[0]
    
    for location, (lat, lon) in coords.items():
        distance = haversine_distance(site_lat, site_lon, lat, lon)
        
        # Draw a line between site and proximity
        folium.PolyLine(
            locations=[[site_lat, site_lon], [lat, lon]],
            color='gray',
            weight=1.5,
            opacity=0.5
        ).add_to(proximity_map)
        
        # Add proximity marker
        folium.Marker(
            location=[lat, lon],
            popup=f"<b>{location}</b><br>Distance to {site}: {distance:.2f} km",
            icon=folium.Icon(color='blue', icon='city', prefix='fa')
        ).add_to(proximity_map)

# Display the map
proximity_map

     Launch Site             Proximity  Distance (km)
0    CCAFS LC-40             Coastline       0.579576
1    CCAFS LC-40   Cape Canaveral City      17.609204
2    CCAFS LC-40  Kennedy Space Center       7.610434
3   CCAFS SLC-40             Coastline       0.509584
4   CCAFS SLC-40   Cape Canaveral City      17.715628
5   CCAFS SLC-40  Kennedy Space Center       7.627281
6     KSC LC-39A             Coastline       0.888596
7     KSC LC-39A       Titusville City      16.276893
8     KSC LC-39A  Kennedy Space Center       1.381912
9    VAFB SLC-4E             Coastline       1.450584
10   VAFB SLC-4E           Lompoc City      14.001017
11   VAFB SLC-4E     Vandenberg AFB HQ      15.376579


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [21]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [22]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [23]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
from folium.plugins import MousePosition

# Create a map centered on Florida launch sites
coastline_map = folium.Map(location=[28.563, -80.571], zoom_start=10)

# Add MousePosition to get coordinates
MousePosition(
    position='topright',
    separator=' | ',
    prefix="Coordinates:",
    lat_formatter=":.5f",
    lng_formatter=":.5f"
).add_to(coastline_map)

# Add launch sites
for site in ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A']:
    site_data = spacex_df[spacex_df['Launch Site'] == site].iloc[0]
    folium.Marker(
        location=[site_data['Lat'], site_data['Long']],
        popup=f"<b>{site}</b>",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(coastline_map)

coastline_map
# Pre-identified closest coastline points (from MousePosition)
coastline_points = {
    "CCAFS LC-40": (28.56367, -80.57163),
    "CCAFS SLC-40": (28.56367, -80.57163),  # Same as LC-40
    "KSC LC-39A": (28.58010, -80.64220),
    "VAFB SLC-4E": (34.63290, -120.62660)  # Vandenberg
}

# Calculate distances
coastline_distances = []
for site in spacex_df['Launch Site'].unique():
    site_data = spacex_df[spacex_df['Launch Site'] == site].iloc[0]
    coast_lat, coast_lon = coastline_points[site]
    
    distance = haversine_distance(
        site_data['Lat'], site_data['Long'],
        coast_lat, coast_lon
    )
    
    coastline_distances.append({
        'Launch Site': site,
        'Coastline Coordinates': f"{coast_lat:.5f}, {coast_lon:.5f}",
        'Distance to Coast (km)': round(distance, 2)
    })

# Display as a table
pd.DataFrame(coastline_distances)
# Create a new map with coastline markers
final_map = folium.Map(location=[28.563, -80.571], zoom_start=5)

# Add launch sites (red) and coastlines (blue)
for site in spacex_df['Launch Site'].unique():
    site_data = spacex_df[spacex_df['Launch Site'] == site].iloc[0]
    coast_lat, coast_lon = coastline_points[site]
    
    # Launch site marker
    folium.Marker(
        location=[site_data['Lat'], site_data['Long']],
        popup=f"<b>{site}</b>",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(final_map)
    
    # Coastline marker
    folium.Marker(
        location=[coast_lat, coast_lon],
        popup=f"<b>Closest Coastline</b><br>Distance: {haversine_distance(site_data['Lat'], site_data['Long'], coast_lat, coast_lon):.2f} km",
        icon=folium.Icon(color='blue', icon='water', prefix='fa')
    ).add_to(final_map)
    
    # Connecting line
    folium.PolyLine(
        locations=[[site_data['Lat'], site_data['Long']], [coast_lat, coast_lon]],
        color='gray',
        weight=1.5,
        dash_array='5,5'
    ).add_to(final_map)

final_map

In [24]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )
# Create a map centered on Florida launch sites
distance_map = folium.Map(location=[28.563, -80.571], zoom_start=10)

# Define coastline points (from previous identification)
coastline_points = {
    "CCAFS LC-40": (28.56367, -80.57163),
    "CCAFS SLC-40": (28.56367, -80.57163),
    "KSC LC-39A": (28.58010, -80.64220),
    "VAFB SLC-4E": (34.63290, -120.62660)
}

# Add launch sites and distance markers
for site in spacex_df['Launch Site'].unique():
    site_data = spacex_df[spacex_df['Launch Site'] == site].iloc[0]
    coast_lat, coast_lon = coastline_points[site]
    
    # Calculate distance
    distance = haversine_distance(
        site_data['Lat'], site_data['Long'],
        coast_lat, coast_lon
    )
    
    # Add launch site marker
    folium.Marker(
        location=[site_data['Lat'], site_data['Long']],
        popup=f"<b>{site}</b>",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(distance_map)
    
    # Add coastline point with distance display
    distance_marker = folium.Marker(
        location=[coast_lat, coast_lon],
        icon=folium.DivIcon(
            icon_size=(150, 30),
            icon_anchor=(0, 0),
            html=f"""<div style="font-size: 12pt; color: #d35400; 
                      background: white; padding: 3px; border-radius: 3px;
                      border: 1px solid gray;">
                      <b>{distance:.2f} KM</b>
                    </div>"""
        )
    )
    distance_marker.add_to(distance_map)
    
    # Add connecting line
    folium.PolyLine(
        locations=[[site_data['Lat'], site_data['Long']], [coast_lat, coast_lon]],
        color='blue',
        weight=1.5,
        dash_array='5,5',
        opacity=0.7
    ).add_to(distance_map)

# Add title to the map
title_html = '''
    <h3 align="center" style="font-size:16px">
    <b>Distance Between Launch Sites and Nearest Coastline</b>
    </h3>
'''
distance_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
distance_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [26]:
# Create a new map with better visualization
site_map = folium.Map(location=[28.563, -80.571], zoom_start=10)

# Define coastline points (previously identified)
coastline_points = {
    "CCAFS LC-40": (28.56367, -80.57163),
    "CCAFS SLC-40": (28.56367, -80.57163),
    "KSC LC-39A": (28.58010, -80.64220),
    "VAFB SLC-4E": (34.63290, -120.62660)
}

# Custom style for the Polylines
line_style = {
    'color': '#3388ff',
    'weight': 3,
    'dashArray': '5, 5',
    'opacity': 0.7
}

# Add each launch site and its coastline connection
for site in spacex_df['Launch Site'].unique():
    site_data = spacex_df[spacex_df['Launch Site'] == site].iloc[0]
    coast_lat, coast_lon = coastline_points[site]
    
    # Calculate distance
    distance = haversine_distance(
        site_data['Lat'], site_data['Long'],
        coast_lat, coast_lon
    )
    
    # Create coordinates for the PolyLine
    coordinates = [
        [site_data['Lat'], site_data['Long']],
        [coast_lat, coast_lon]
    ]
    
    # Create and add the PolyLine
    lines = folium.PolyLine(
        locations=coordinates,
        **line_style,
        tooltip=f"Distance: {distance:.2f} km"
    )
    site_map.add_child(lines)
    
    # Add launch site marker
    folium.Marker(
        [site_data['Lat'], site_data['Long']],
        popup=f"<b>{site}</b>",
        icon=folium.Icon(color='red', icon='rocket', prefix='fa')
    ).add_to(site_map)
    
    # Add coastline marker with distance label
    folium.Marker(
        [coast_lat, coast_lon],
        icon=folium.DivIcon(
            icon_size=(150,30),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color: white; background: #3388ff; padding: 3px; border-radius: 3px;"><b>{distance:.2f} km</b></div>'
        )
    ).add_to(site_map)

# Add layer control and title
folium.LayerControl().add_to(site_map)
title_html = '''
    <h3 align="center" style="font-size:16px">
    <b>Launch Site to Coastline Distance Visualization</b>
    </h3>
'''
site_map.get_root().html.add_child(folium.Element(title_html))

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
